![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to analyze car rental customer satisfaction and offer recommendation.

**Note:** Please note that for the watsonx challenge, please run these notebooks in IBM Cloud and not on on your laptop/desktop.

This notebook contains the steps and code to demonstrate support of text sentiment analysis in Watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

<a id="setup"></a>
## Set up the environment

### Install and import the dependecies

In [76]:
!pip install datasets | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install ibm-watson-machine-learning==1.0.312 | tail -n 1

**Note:** Please restart the notebook kernel to pick up proper version of packages installed above.

In [77]:
import os, getpass
from pandas import read_csv

### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. Instructions have been provided to generate IBM Cloud API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [78]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

Please enter your api key (hit enter): ········


### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. When you run notebook on IBM Cloud, project in which it runs is saved as environment variable PROJECT_ID.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [79]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [80]:
filename_test = 'https://watsonx-gsi-challenge.s3.jp-tok.cloud-object-storage.appdomain.cloud/track1/test.csv'
filename_train = 'https://watsonx-gsi-challenge.s3.jp-tok.cloud-object-storage.appdomain.cloud/track1/train.csv'

test_data = read_csv(filename_test)
train_data = read_csv(filename_train)

In [81]:
train_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2944,Female,M,2,41.92,Active,No,Customer service was friendly and helpful.,1,Service: Knowledge,NaN
1,1119,Female,M,2,33.60,Active,Yes,Customer service was good at MSP airport and t...,1,Service: Knowledge,NaN
2,0,Male,M,0,51.00,Inactive,Yes,I do not understand why I have to pay additio...,0,Product: Pricing and Billing,Premium features
3,1085,Female,S,2,42.00,Inactive,No,Based on the customer service personnel I enco...,0,Service: Attitude,On-demand pickup location
4,0,Female,M,2,44.10,Active,No,Provide more convenient car pickup from the ai...,0,Service: Orders/Contracts,On-demand pickup location


In [82]:
test_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2771,Female,M,2,49.99,Inactive,No,"last time I rented a car was at Manchester, NH...",0,Product: Functioning,On-demand pickup location
1,1133,Male,S,1,56.05,Inactive,No,Please lower the prices.,0,Product: Pricing and Billing,Free Upgrade
2,900,Female,M,1,64.64,Active,No,Excellent response dealing with child seat.,1,Service: Accessibility,NaN
3,3795,Male,M,0,46.51,Inactive,No,"all went quite smoothly... it was Enterprise, ...",1,Service: Accessibility,NaN
4,3541,Male,S,1,17.01,Inactive,Yes,"Slow, long lineup",0,Product: Functioning,On-demand pickup location


<a id="models"></a>
## Foundation Models on Watsonx



Below code invokes Watson Machine Learning API to invoke Watsonx.ai LLMs


In [83]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

<a id="predict"></a>
## Evaluate the model, prompt and parameters

### **1. Customer satisfaction**

Define instructions for the model to recognize if customer was satisfied or unsatisfied.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [105]:
satisfaction_instruction = """
Understand the sentiment of the comment and Classify the customer satisfaction as 0 or 1 and classify as 1 if the sentiment seems neutral.
 
comment: I have had a few recent rentals that have taken a very very long time, with no offer of apology.
In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available.
satisfaction:0
comment:they were trying to satisfy me even though my wife tells me I was being rude
satisfaction:1
comment:The customer service agent was helpful, but I find that providing all of the information to the agent takes too long.  It would be nice if I could provide all of the information to the agent prior to coming the rental place.
satisfaction:0
comment:Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.	0
Provide more convenient car pickup from the airport parking.
satisfaction:0
comment:We had a small problem with the size of car we reserved being available. The company was able to send another car over from a different location. We were delayed in getting on the road, but were satisfied with the car.
satisfaction:1
comment:I guess it really DOES try harder.
satisfaction:1
comment:It was okay, we got the car quickly, which is the most important thing
satisfaction:1
comment:Customer service was average.  They did not go above and beyond.  They did, however, do more than the minimum.
satisfaction:1
comment:My experience was positive .The thing I didnt like was returning the car with full tank.  It was time consuming, but I didn't want to pay fill-up charge to the rental company
satisfaction:1
comment:Person very friendly but only person working counter
satisfaction:1
comment:Last time I rented a car was when I went skiing with my whole family.  We got a Chevy Blazer.  We didn't think it was as large as a Ford Explorer, so we asked to switch.  The agent was very nice and gave us the Ford Explorer.
satsifaction:1
"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [106]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 10,
    "min_new_tokens": 1,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v1"

Analyze the customer satisfaction for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [107]:
results = []
prompt = Prompt(access_token, project_id)
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    results.append(prompt.generate(" ".join([satisfaction_instruction, input_text,"satisfaction:"]), model_id, parameters))

### Calculate the F1 micro score

In [108]:
from sklearn.metrics import f1_score

print('f1_micro_score', f1_score(satisfaction, results, average='micro'))

f1_micro_score 0.98


### **2. Offer Recommendation**

Define instructions for the model to recommend best offer to an unsatisfied customer.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [88]:
offer_recommendation_instruction = """
Identify the customer's sentiment in the comment.
Identify the main pain point: What is the customer most unhappy with?
Recommend an offer that addresses the customer's pain point.
Below are the 4 available offers.

1.On-demand pickup location
When to offer: For customers who complained about slow, long lineup, inconvenient pickup locations.
Who don't like customers yelling employees not in charge of the issues.
When windows were closed. Hate having to walk with big bags and search the car which is always in the furthest corner.
When customers are in a hurry, such as those who have a flight to catch or who have an important meeting to attend.
When customer is unhappy with the wait time to pick up their car. Bad customer service(rep) experience, overworked employees, bad printer or register.

2.Free Upgrade
When to offer: This offer should be given to customers who have a lot of luggage, a negative experience with the company, such as those who have experienced slow service, hate to wait long lines(old customer),rude customer service, or problems with their rental car.
This offer can also be given to customers who have been loyal to the company, such as those who have been renting cars from the company for a long time(many years ago) or who have rented multiple cars from the company in the past(many years ago).
The customer is unhappy with the car they were given.
When a customer asks for lower price or upgrade.
Polite enough customers, who have not spoken to a car rental company recently or last called years ago, who hate to wait in lines and have a lot of luggage.

3.Voucher
When to offer: This offer should be given to customers who have had a very negative or bad experience with the company, received a dirty car, or had problems with their rental car that were not resolved, or overcharged.
For customers who are new to the company, in order to encourage them to rent a car from the company in the future.
The customer had a negative or bad experience(pain) overall and wants to be compensated.
Forget to ask customer details like driver's license. Felt pain with overcharge and got money back.
Customer who looks for enjoyment of the car. Rent a car from airport cause it's cheaper. Who won't return if it's a bad experience.


4.Premium features
When to offer: This offer should be given to customers who have complained about the lack of features in their rental car, such as those who have rented an economy car and wished they had upgraded to a car with more features. Frequent user.
This offer can also be given to customers who are willing to pay extra for additional features, such as a GPS system.
The customer is looking for importance for enjoyment of the car, services.
Agents take insurance and ask customer to pay more even after customer said no.

Examples:

comment:Initially the representative handled the situation badly.  No car was available, even next day.  Then the rep helped to find us a car in another agency.
offer recommended:Free Upgrade
comment:Provide more convenient car pickup from the airport parking.
offer recommended:On-demand pickup location
comment:It would be nice if they included maps to the airport drop off with the car, last time we got lost returning the car and almost missed the flight.  There was nothing on the exit that indicated rental returns.  Actually the one I really liked, was when we
offer recommended:Premium features
comment:It was absolutely ATROCIOUS! My wife and I were in a foreign country  when we realized that our car had an expired license plate and expired proof of insurance!
offer recommended:Voucher

comment:Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
offer recommended:On-demand pickup location
comment:it seems that they never have the class of car that I ordered.  also, the last time I got there earlier than I thought and they wouldn't let me have the car until the time that I had requested it because their computer wouldn't allow it.
offer recommended:Free Upgrade
comment:"sorely lacking"  helpdesk was not much helpfull
offer recommended:Voucher
comment:I would like the reps be knowledgeable about the immediate area around the rental agency and or have maps for the area available free of charge.
offer recommended:Premium features

comment:I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
offer recommended:Premium features
comment:They were too pushy in trying to sell insurance.
offer recommended:Free Upgrade
comment:The customer service agent was helpful, but I find that providing all of the information to the agent takes too long.  It would be nice if I could provide all of the information to the agent prior to coming the rental place.
offer recommended:On-demand pickup location
comment:I have had a few recent rentals that have taken a very very long time, with no offer of apology.  In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just be
offer recommended:Voucher

comment:Customer Service at Alamo was RUDE. We waited in line for over an hour to get our car. Car was bad. Had to change it few days later. Another family traveling with us was upgraded to a larger car as they were out of the type of car booked.
offer recommended:Free Upgrade
comment:what customer service? It was a nightmare
offer recommended:Voucher
comment:I would like the personnel to pretend they care about customer, at least.
offer recommended:On-demand pickup location
comment:States have different rules and regulations pertaining to speed, radar detectors, even when light should be on.  Customer services reps should have a handout or be knowledgeable of a particular state's laws.
offer recommended:Premium features
"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [89]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 30,
    "min_new_tokens": 1,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v1"

Filter test data for unsatisfied customer

In [90]:
unsatisfied_test_data = test_data.loc[test_data['Satisfaction'] == 0]
unsatisfied_test_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2771,Female,M,2,49.99,Inactive,No,"last time I rented a car was at Manchester, NH...",0,Product: Functioning,On-demand pickup location
1,1133,Male,S,1,56.05,Inactive,No,Please lower the prices.,0,Product: Pricing and Billing,Free Upgrade
4,3541,Male,S,1,17.01,Inactive,Yes,"Slow, long lineup",0,Product: Functioning,On-demand pickup location
5,2608,Female,S,0,32.02,Active,No,Customer is important for the enjoyment of the...,0,Product: Functioning,Voucher
7,3382,Male,M,1,52.15,Inactive,No,They should upgrade me every time.,0,Service: Knowledge,Free Upgrade


Analyze the recommended actions for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [91]:
results = []
prompt = Prompt(access_token, project_id)
comments = list(unsatisfied_test_data.Customer_Service)
offer_recommended = list(unsatisfied_test_data.Action.astype(str))

for input_text in comments:
    results.append(prompt.generate(" ".join([offer_recommendation_instruction,"\ncomment:", input_text,"\noffer_recommended:"]), model_id, parameters))

### Calculate the F1 micro score

In [92]:
from sklearn.metrics import f1_score

print('f1_micro_score', f1_score(offer_recommended, results, average='micro'))

f1_micro_score 0.7222222222222222


---

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.